In [1]:
# sudo service mysql stop
# docker run --name mysql_call -p 3306:3306 -e MYSQL_ROOT_PASSWORD=your_secret_pwd -d mysql:latest

In [66]:
import pymysql
import pandas as pd
import numpy as np
from datetime import datetime
import re
#import datetime

In [2]:
with open('mysql_local.pass','r') as file:
    mysql_pass = file.read().replace('\n', '')
    file.close()

In [3]:
conn = pymysql.connect(
            host = '10.2.4.87', 
            user = 'root',
            passwd = mysql_pass,
            db = 'mysql',
            autocommit = True
        )

In [4]:
cursor = conn.cursor()

In [40]:
query = "create database 1c;"
cursor.execute(query)

1

In [5]:
query = "use 1c;"
cursor.execute(query)
for row in cursor.fetchall():
    print(row)

In [6]:
query = "show tables;"
cursor.execute(query)
for row in cursor.fetchall():
    print(row)

('calls',)


In [32]:
#--create table users ( id int not null auto_increment, token varchar(36), 
#email varchar(64), event_date DATETIME, primary key (id) );
query = "create table calls"
query += " ("
query += " id int not null auto_increment,"
query += " call_date DATETIME,"
query += " ak BOOLEAN,"
query += " miko BOOLEAN,"
query += " mrm BOOLEAN,"
query += " incoming BOOLEAN,"
query += " linkedid varchar(64),"
query += " base_name varchar(64),"
query += " primary key (id)"
query += " );"
cursor.execute(query)
for row in cursor.fetchall():
    print(row)

In [31]:
query = "drop table calls;"
cursor.execute(query)
for row in cursor.fetchall():
    print(row)

In [84]:
query = "delete from calls;"
cursor.execute(query)
for row in cursor.fetchall():
    print(row)

In [87]:
query = "SELECT * from calls;"
calls = pd.read_sql(query, con = conn)
calls

,id,call_date,ak,miko,mrm,incoming,linkedid,base_name
0,7255,2021-04-22 07:18:32,0,1,0,0,1619065299.3050508,1c_service_region
1,7256,2021-04-22 07:24:41,1,0,1,0,1619065479.22323747,1c_service_region
2,7257,2021-04-22 07:49:02,0,1,0,0,1619067128.3050609,1c_service_region
3,7258,2021-04-22 07:51:29,0,1,0,0,1619067276.3050641,1c_service_region
4,7259,2021-04-22 08:01:26,0,1,0,0,1619067872.3050737,1c_service_region
...,...,...,...,...,...,...,...,...
10654,17909,2021-04-22 21:36:05,0,1,0,0,1619116712.3074504,1c_service_spb
10655,17910,2021-04-22 21:40:50,0,1,0,1,1619116992.3074538,1c_service_spb
10656,17911,2021-04-22 21:53:17,0,1,0,1,1619117737.3074547,1c_service_spb
10657,17912,2021-04-22 22:14:15,0,1,0,1,1619118984.3074555,1c_service_spb


In [79]:
calls.linkedid = calls.linkedid.str.replace('.WAV','')

In [81]:
calls[calls.linkedid=='1619120428.22353330.WAV']

,id,call_date,ak,miko,mrm,incoming,linkedid,base_name


In [71]:
def get_base_name(val):
    return re.findall( r'"(.*?)"', val)[1]

In [73]:
df.base_name = df.base_name.apply(get_base_name)

In [74]:
df

,call_date,ak,miko,mrm,incoming,linkedid,base_name
0,2021-04-22 07:18:32,False,True,False,False,1619065299.3050508,1c_service_region
1,2021-04-22 07:24:41,True,False,True,False,1619065479.22323747.WAV,1c_service_region


In [70]:
df.base_name = df.base_name.str.replace(r'"(.*?)"', '')

,call_date,ak,miko,mrm,incoming,linkedid,base_name
0,2021-04-22 07:18:32,False,True,False,False,1619065299.3050508,"Srvr=""msk-srv1c01"" Ref=""1c_service_region"""
1,2021-04-22 07:24:41,True,False,True,False,1619065479.22323747.WAV,"Srvr=""msk-srv1c01"" Ref=""1c_service_region"""


In [69]:
word = 'Srvr="msk-srv1c01" Ref="1c_service_region"'
re.findall( r'"(.*?)"', word)[1]

'1c_service_region'

In [67]:
re.findall( r'all (.*?) are', 'all cats are smarter than dogs, all dogs are dumber than cats')
# Output: ['cats', 'dogs']

[x.group() for x in re.finditer( r'all (.*?) are', 'all cats are smarter than dogs, all dogs are dumber than cats')]
# Output: ['all cats are', 'all dogs are']

['all cats are', 'all dogs are']

In [62]:
query = "SELECT column_name FROM information_schema.columns WHERE table_name='calls';"
calls = pd.read_sql(query, con = conn)
calls

,COLUMN_NAME
0,id
1,call_date
2,ak
3,miko
4,mrm
5,incoming
6,linkedid
7,base_name


In [34]:
query = "SELECT column_name FROM information_schema.columns WHERE table_name='calls';"
calls = pd.read_sql(query, con = conn)
calls

,COLUMN_NAME
0,id
1,call_date
2,ak
3,miko
4,mrm
5,incoming
6,linkedid
7,base_name


In [57]:
filename = 'data.csv'
dateparser = lambda x: datetime.strptime(x, "%d.%m.%Y %H:%M:%S")
dtypes = {
        'call_date': 'datetime64',
        'ak': 'bool',
        'miko': 'bool',
         'mrm': 'bool',
         'incoming': 'bool',
         'linkedid': 'str',
         'base_name': 'str',
     }
#df = pd.read_csv(filename, ';', dtype=dtypes, parse_dates = True)
#df = pd.read_csv(filename, ';', parse_dates = True)
df = pd.read_csv(filename, ';', parse_dates = ['call_date'], date_parser=dateparser)
df.call_date.iloc[0], type(df.call_date.iloc[0])

(Timestamp('2021-04-22 07:18:32'), pandas._libs.tslibs.timestamps.Timestamp)

In [60]:
df.base_name.iloc[0], type(df.base_name.iloc[0])

('Srvr="msk-srv1c01" Ref="1c_service_region" ', str)